In [4]:
import scipy
import numpy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from matplotlib import pyplot as plt
from IPython.core.pylabtools import figsize

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns

In [5]:
# Create a 3D input
def create_dataset (X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X)-time_steps):
        v = X[i:i+time_steps, :]
        Xs.append(v)
        ys.append(y[i+time_steps])
    return np.array(Xs), np.array(ys)

# Function for LSTM and GRU model
def create_model(units, m, learning_rate):
    model = Sequential()
    model.add(m (units = units, return_sequences = True,
                input_shape = [X_train.shape[1], X_train.shape[2]], 
               activation='relu'))
    model.add(Dropout(0.2))
    
    initializer = tf.keras.initializers.GlorotUniform()
    model.add(m (units = units, kernel_initializer=initializer, 
               activation='relu'))
    
    model.add(Dropout(0.2))
    model.add(Dense(units = 1, activation = "relu"))
    
    opt = keras.optimizers.Adam(learning_rate= learning_rate)
    #Compile model
    model.compile(loss='mae', optimizer= opt)
    return model

# Fitting LSTM and GRU
def fit_model(model):
    earlystopping  = keras.callbacks.EarlyStopping(monitor = 'val_loss',
   
    #The patience for early stopping is set to 20 as the validation and training loss could zig zag.
                                               patience = 20)
    #Epochs at 200 is simply a maximum in case the model keeps running with continued improvement.
    #Early stopping is meant to prevent overfitting.
    history = model.fit(X_train, y_train, epochs = 200,  
                        validation_split = 0.2,
                        shuffle = True, callbacks = [earlystopping],
                        verbose = 1)
    return history
# Plot train loss and validation loss
def plot_loss (history):
    plt.figure(figsize = (10, 6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('Loss')
    plt.xlabel('epoch')
    plt.legend(['Train loss', 'Validation loss'], loc='upper right')
    
# Make prediction
def prediction(model):
    prediction = model.predict(X_test)
    prediction = scaler_y.inverse_transform(prediction)
    return prediction


# Plot true future vs prediction
def plot_future(prediction, y_test):
    plt.figure(figsize=(10, 6))
    range_future = len(prediction)
        
    plt.plot(np.arange(range_future), np.array(y_test), 
             label='True Future')
    
    plt.plot(np.arange(range_future),np.array(prediction),
            label='Prediction')
    
    plt.legend(loc='upper left')
    plt.xlabel('Time (hour)')
    plt.ylabel('Bike users per time period') 
    
# Define a function to calculate MAE and RMSE
def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    print(model_name + ':')
    print('Mean Absolute Error: {:.4f}'.format(mae))
    print('Root Mean Square Error: {:.4f}'.format(rmse))
    print('')    

In [6]:
#Warning, running this will take quite some time

timesteps = [1, 12, 24]
learning_rate = [0.01, 0.001, 0.0001]
units = [16, 32, 64, 128]

column_names = ["Time step", "Learning rate","Units", "GRU RMSE", "GRU MAE","GRU R-Squared", "LSTM RMSE", "LSTM MAE", "LSTM R-Squared"]
metrics = pd.DataFrame(columns = column_names)

for g in timesteps:
    for k in learning_rate:
        for u in units:
            GRU_RMSE = []
            GRU_MAE = []
            GRU_rsquared = []

            LSTM_RMSE = []
            LSTM_MAE = []
            LSTM_rsquared = []

            repeats = 3
            for i in range (repeats):
                df = pd.read_csv("Final_Thesis_Dataset.csv", index_col = 0)

                train_size = int(len(df)*0.75)
                train_dataset, test_dataset = df.iloc[:train_size], df.iloc[train_size:]

                # Split train data to X and y
                X_train = train_dataset.drop(['Users'], axis = 1)
                y_train = train_dataset.loc[:,['Users']]

                # Split test data to X and y
                X_test = test_dataset.drop(['Users'], axis = 1)
                y_test = test_dataset.loc[:,['Users']]

                # Different scaler for input and output
                scaler_x = MinMaxScaler(feature_range = (0,1))
                scaler_y = MinMaxScaler(feature_range = (0,1))

                # Fit the scaler using available training data
                input_scaler = scaler_x.fit(X_train)
                output_scaler = scaler_y.fit(y_train)

                # Apply the scaler to training data
                train_y_norm = output_scaler.transform(y_train)
                train_x_norm = input_scaler.transform(X_train)

                # Apply the scaler to test data
                test_y_norm = output_scaler.transform(y_test)
                test_x_norm = input_scaler.transform(X_test)

                
                TIME_STEPS = g
                X_test, y_test = create_dataset(test_x_norm, test_y_norm,   
                                                TIME_STEPS)
                X_train, y_train = create_dataset(train_x_norm, train_y_norm, 
                                                  TIME_STEPS)

                # GRU and LSTM
                model_gru = create_model(u, GRU, k)
                model_lstm = create_model(u, LSTM, k)

                history_lstm = fit_model(model_lstm)
                history_gru = fit_model(model_gru)


                y_test = scaler_y.inverse_transform(y_test)
                y_train = scaler_y.inverse_transform(y_train)

                prediction_lstm = prediction(model_lstm)
                prediction_gru = prediction(model_gru)

                errors_gru = prediction_gru - y_test
                errors_lstm = prediction_lstm - y_test



                GRU_RMSE.append(np.sqrt(np.square(errors_gru).mean()))
                GRU_MAE.append(np.abs(errors_gru).mean())
                GRU_rsquared.append(r2_score(y_test, prediction_gru))
                LSTM_RMSE.append(np.sqrt(np.square(errors_lstm).mean()))
                LSTM_MAE.append(np.abs(errors_lstm).mean())
                LSTM_rsquared.append(r2_score(y_test, prediction_lstm))
        
            GRU_RMSE_values = round(np.mean(GRU_RMSE), 2)
            GRU_MAE_values = round(np.mean(GRU_MAE), 2)
            GRU_rsquared_values = round(np.mean(GRU_rsquared), 2)

            LSTM_RMSE_values = round(np.mean(LSTM_RMSE), 2)
            LSTM_MAE_values = round(np.mean(LSTM_MAE), 2)
            LSTM_rsquared_values = round(np.mean(LSTM_rsquared), 2)

            metrics.loc[len(metrics)] = [g, k, u, GRU_RMSE_values, GRU_MAE_values,GRU_rsquared_values, 
                                         LSTM_RMSE_values, LSTM_MAE_values, LSTM_rsquared_values]

Epoch 1/200
102/102 [==============================] - 1s 8ms/step - loss: 0.1244 - val_loss: 0.0706
Epoch 2/200
102/102 [==============================] - 0s 1ms/step - loss: 0.0682 - val_loss: 0.0608
Epoch 3/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0597 - val_loss: 0.0570
Epoch 4/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0570 - val_loss: 0.0549
Epoch 5/200
102/102 [==============================] - 0s 1ms/step - loss: 0.0561 - val_loss: 0.0600
Epoch 6/200
102/102 [==============================] - 0s 1ms/step - loss: 0.0558 - val_loss: 0.0561
Epoch 7/200
102/102 [==============================] - 0s 1ms/step - loss: 0.0532 - val_loss: 0.0561
Epoch 8/200
102/102 [==============================] - 0s 1ms/step - loss: 0.0528 - val_loss: 0.0521
Epoch 9/200
102/102 [==============================] - 0s 1ms/step - loss: 0.0526 - val_loss: 0.0553
Epoch 10/200
102/102 [==============================] - 0s 1ms/step - loss: 0.0523 - val_lo

102/102 [==============================] - 0s 1ms/step - loss: 0.0584 - val_loss: 0.0467
Epoch 5/200
102/102 [==============================] - 0s 1ms/step - loss: 0.0560 - val_loss: 0.0481
Epoch 6/200
102/102 [==============================] - 0s 1ms/step - loss: 0.0554 - val_loss: 0.0466
Epoch 7/200
102/102 [==============================] - 0s 1ms/step - loss: 0.0545 - val_loss: 0.0506
Epoch 8/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0539 - val_loss: 0.0530
Epoch 9/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0531 - val_loss: 0.0509
Epoch 10/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0515 - val_loss: 0.0511
Epoch 11/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0514 - val_loss: 0.0438
Epoch 12/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0510 - val_loss: 0.0508
Epoch 13/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0498 - val_loss: 0.044

102/102 [==============================] - 0s 2ms/step - loss: 0.0483 - val_loss: 0.0522
Epoch 18/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0500 - val_loss: 0.0598
Epoch 19/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0489 - val_loss: 0.0515
Epoch 20/200
102/102 [==============================] - 0s 1ms/step - loss: 0.0487 - val_loss: 0.0575
Epoch 21/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0479 - val_loss: 0.0595
Epoch 22/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0481 - val_loss: 0.0543
Epoch 23/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0482 - val_loss: 0.0519
Epoch 24/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0480 - val_loss: 0.0578
Epoch 25/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0475 - val_loss: 0.0528
Epoch 26/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0480 - val_loss: 

102/102 [==============================] - 0s 2ms/step - loss: 0.0430 - val_loss: 0.0529
Epoch 35/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0423 - val_loss: 0.0533
Epoch 36/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0430 - val_loss: 0.0522
Epoch 37/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0430 - val_loss: 0.0514
Epoch 38/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0433 - val_loss: 0.0550
Epoch 1/200
102/102 [==============================] - 1s 7ms/step - loss: 0.0955 - val_loss: 0.0480
Epoch 2/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0631 - val_loss: 0.0449
Epoch 3/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0577 - val_loss: 0.0470
Epoch 4/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0579 - val_loss: 0.0503
Epoch 5/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0531 - val_loss: 0.047

102/102 [==============================] - 0s 2ms/step - loss: 0.0499 - val_loss: 0.0491
Epoch 12/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0489 - val_loss: 0.0514
Epoch 13/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0483 - val_loss: 0.0579
Epoch 14/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0478 - val_loss: 0.0528
Epoch 15/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0488 - val_loss: 0.0496
Epoch 16/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0466 - val_loss: 0.0508
Epoch 17/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0471 - val_loss: 0.0513
Epoch 18/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0473 - val_loss: 0.0537
Epoch 19/200
102/102 [==============================] - 0s 3ms/step - loss: 0.0467 - val_loss: 0.0493
Epoch 20/200
102/102 [==============================] - 0s 2ms/step - loss: 0.0466 - val_loss: 

...

102/102 [==============================] - 1s 14ms/step - loss: 0.0486 - val_loss: 0.0378
Epoch 120/200
102/102 [==============================] - 1s 14ms/step - loss: 0.0472 - val_loss: 0.0362
Epoch 121/200
102/102 [==============================] - 1s 14ms/step - loss: 0.0471 - val_loss: 0.0366
Epoch 122/200
102/102 [==============================] - 2s 15ms/step - loss: 0.0473 - val_loss: 0.0379
Epoch 123/200
102/102 [==============================] - 2s 16ms/step - loss: 0.0476 - val_loss: 0.0388
Epoch 124/200
102/102 [==============================] - 2s 16ms/step - loss: 0.0472 - val_loss: 0.0379
Epoch 125/200
102/102 [==============================] - 2s 15ms/step - loss: 0.0470 - val_loss: 0.0369
Epoch 126/200
102/102 [==============================] - 1s 14ms/step - loss: 0.0466 - val_loss: 0.0357
Epoch 127/200
102/102 [==============================] - 1s 15ms/step - loss: 0.0473 - val_loss: 0.0369
Epoch 128/200
102/102 [==============================] - 1s 14ms/step - loss: 

102/102 [==============================] - 2s 20ms/step - loss: 0.0509 - val_loss: 0.0392
Epoch 39/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0509 - val_loss: 0.0403
Epoch 40/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0505 - val_loss: 0.0353
Epoch 41/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0503 - val_loss: 0.0375
Epoch 42/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0503 - val_loss: 0.0358
Epoch 43/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0503 - val_loss: 0.0387
Epoch 44/200
102/102 [==============================] - 2s 19ms/step - loss: 0.0496 - val_loss: 0.0400
Epoch 45/200
102/102 [==============================] - 2s 19ms/step - loss: 0.0494 - val_loss: 0.0378
Epoch 46/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0498 - val_loss: 0.0357
Epoch 47/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0491 - 

102/102 [==============================] - 2s 24ms/step - loss: 0.0496 - val_loss: 0.0669
Epoch 58/200
102/102 [==============================] - 2s 24ms/step - loss: 0.0503 - val_loss: 0.0646
Epoch 59/200
102/102 [==============================] - 2s 24ms/step - loss: 0.0498 - val_loss: 0.0588
Epoch 60/200
102/102 [==============================] - 2s 24ms/step - loss: 0.0512 - val_loss: 0.0629
Epoch 61/200
102/102 [==============================] - 2s 24ms/step - loss: 0.0502 - val_loss: 0.0642
Epoch 62/200
102/102 [==============================] - 2s 24ms/step - loss: 0.0488 - val_loss: 0.0599
Epoch 63/200
102/102 [==============================] - 2s 23ms/step - loss: 0.0494 - val_loss: 0.0654
Epoch 64/200
102/102 [==============================] - 2s 24ms/step - loss: 0.0493 - val_loss: 0.0578
Epoch 65/200
102/102 [==============================] - 2s 24ms/step - loss: 0.0491 - val_loss: 0.0599
Epoch 66/200
102/102 [==============================] - 2s 24ms/step - loss: 0.0488 - 

102/102 [==============================] - 2s 23ms/step - loss: 0.0472 - val_loss: 0.0439
Epoch 68/200
102/102 [==============================] - 2s 22ms/step - loss: 0.0467 - val_loss: 0.0450
Epoch 69/200
102/102 [==============================] - 2s 21ms/step - loss: 0.0469 - val_loss: 0.0424
Epoch 70/200
102/102 [==============================] - 2s 21ms/step - loss: 0.0472 - val_loss: 0.0443
Epoch 71/200
102/102 [==============================] - 2s 21ms/step - loss: 0.0464 - val_loss: 0.0440
Epoch 72/200
102/102 [==============================] - 2s 22ms/step - loss: 0.0456 - val_loss: 0.0413
Epoch 73/200
102/102 [==============================] - 2s 21ms/step - loss: 0.0466 - val_loss: 0.0455
Epoch 74/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0469 - val_loss: 0.0428
Epoch 75/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0462 - val_loss: 0.0425
Epoch 76/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0461 - 

Epoch 25/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0568 - val_loss: 0.0419
Epoch 26/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0567 - val_loss: 0.0413
Epoch 27/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0553 - val_loss: 0.0399
Epoch 28/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0551 - val_loss: 0.0436
Epoch 29/200
102/102 [==============================] - 2s 21ms/step - loss: 0.0548 - val_loss: 0.0417
Epoch 30/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0537 - val_loss: 0.0411
Epoch 31/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0529 - val_loss: 0.0414
Epoch 32/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0531 - val_loss: 0.0412
Epoch 33/200
102/102 [==============================] - 2s 20ms/step - loss: 0.0526 - val_loss: 0.0399
Epoch 34/200
102/102 [==============================] - 2s 20ms/step - lo

In [7]:
#metrics.insert(loc = 0, column = "Time steps", value = timesteps)

In [8]:
metrics

,Time step,Learning rate,Units,GRU RMSE,GRU MAE,GRU R-Squared,LSTM RMSE,LSTM MAE,LSTM R-Squared
0,1.0,0.0100,16.0,24.83,16.68,0.72,28.51,18.50,0.63
1,1.0,0.0100,32.0,25.82,16.70,0.69,26.45,17.53,0.68
2,1.0,0.0100,64.0,27.06,17.65,0.66,26.08,16.94,0.69
3,1.0,0.0100,128.0,26.04,17.54,0.69,26.39,17.42,0.68
4,1.0,0.0010,16.0,23.91,15.80,0.74,24.36,16.07,0.73
5,1.0,0.0010,32.0,25.14,16.61,0.71,25.63,16.82,0.70
6,1.0,0.0010,64.0,25.29,16.42,0.71,25.90,16.96,0.69
7,1.0,0.0010,128.0,24.79,16.50,0.72,25.41,16.77,0.70
8,1.0,0.0001,16.0,24.46,16.31,0.73,24.62,16.29,0.72
9,1.0,0.0001,32.0,24.76,16.46,0.72,24.31,16.20,0.73


In [9]:
#pip install dataframe_image

In [10]:
import dataframe_image as dfi
df_styled = metrics.style.background_gradient().format({
    'Time step': '{:,.0f}'.format,
    'Learning rate': '{:,.4f}'.format,
    'Units': '{:,.0f}'.format,
    'GRU RMSE': '{:,.2f}'.format,
    'GRU MAE': '{:,.2f}'.format,
    'GRU R-Squared': '{:,.2f}'.format,
    'LSTM RMSE': '{:,.2f}'.format,
    'LSTM MAE': '{:,.2f}'.format,
    'LSTM R-Squared': '{:,.2f}'.format
})
dfi.export(df_styled, "hyperparameter_tuning.png")

C:\Users\nuthi\anaconda3\lib\site-packages\dataframe_image\_pandas_accessor.py:69: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  html = '<div>' + obj.render() + '</div>'
